In [84]:
from transformers import QuestionAnsweringPipeline, AutoAdapterModel, AutoModelWithHeads, AutoTokenizer, AutoConfig
from transformers.onnx import OnnxConfig, validate_model_outputs, export
from transformers.models.bert import BertOnnxConfig

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
from onnxruntime import InferenceSession
import onnxruntime

from onnx_opcounter import calculate_params

import os
import time
import torch
import numpy as np

from datasets import load_metric, load_dataset

from typing import Mapping, OrderedDict
from pathlib import Path
import random
import pandas as pd

In [158]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithHeads.from_pretrained("bert-base-uncased")
adapter_name = model.load_adapter("AdapterHub/bert-base-uncased-pf-cosmos_qa", source="hf")
model.active_adapters = adapter_name

def mc_model_inference(question, context, choices):
    outputs = []

    raw_input = [[context, question + " " + choice] for choice in choices]
    inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    answer_idx = torch.argmax(outputs.logits)
    return choices[answer_idx]

question = "What animal has the most hair?"
context = "Fish are typically not hairy. Cats have 10g of hair. Tigers have 12g of hair. Horses have 100g of hair."
answer0 = "Cat"
answer1= "Horse"
answer2= "Tiger"
answer3= "Fish"
choices = [answer0, answer1, answer2, answer3]
answer = mc_model_inference(question, context, choices)
print(answer)

/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/models/bert/adapter_model.py:269: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/models/bert/adapter_model.py:247: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This 

Horse


In [86]:
config = AutoConfig.from_pretrained("bert-base-uncased")
onnx_config = BertOnnxConfig(config, task="multiple-choice")

onnx_path = Path("onnx/cosmosqabert/model.onnx")

onnx_inputs, onnx_outputs = export(tokenizer, model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/layer.py:49: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if context.output_adapter_gating_scores:
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/composition.py:203: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if tensor is not None and hidden_states.shape[0] != tensor.shape[0]:
/home/daedalus/anaconda3/envs/adapter/lib/python3.9/site-packages/transformers/adapters/context.py:117: TracerWarning: Converting a tensor to a Pyt

In [159]:
def onnx_inference(onnx_path, question, context, choices):
    onnx_model = InferenceSession(
        str(onnx_path), providers=["CPUExecutionProvider"]
    )

    raw_input = [[context, question + " " + choice] for choice in choices]
    inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors="np")

    inputs['token_type_ids'] = np.expand_dims(inputs['token_type_ids'], axis=0)
    inputs['input_ids'] =  np.expand_dims(inputs['input_ids'], axis=0)
    inputs['attention_mask'] =  np.expand_dims(inputs['attention_mask'], axis=0)

    outputs = onnx_model.run(input_feed=dict(inputs), output_names=None)

    answer_idx = np.argmax(outputs[0])
    return choices[answer_idx]

answer = onnx_inference(onnx_path, question, context, choices)
print(answer)

[array([[-4.6730275, -2.3068457, -4.9169436, -3.3432305]], dtype=float32)]
Horse
